In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import json
import urllib.parse

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

# MongoDB Authentication
username = "aacuser"
password = "aacuser"
hostname = "localhost"
port = 27017
database = "AAC"
collection = "animals"

# Instantiate the CRUD module with authentication
shelter = AnimalShelter(username, password)



# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({"animal_type": {"$exists": True}}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

app.layout = html.Div([
    # Top Header Section (Logo, Name, Title)
    html.Div([
        html.A(
            html.Img(src='/assets/logo.png', style={'height': '80px', 'margin-right': '20px'}),
            href="https://www.snhu.edu", target="_blank"
        ),
        html.H1('SNHU CS-340 Dashboard', style={'flex-grow': '2', 'text-align': 'center'}),
        html.H4("Developed by: Omer Cengiz", style={'margin-left': '20px'})
    ], style={'display': 'flex', 'align-items': 'center', 'justify-content': 'space-between'}),
    html.Hr(),
    
    # Add Rescue Type Filter
    html.Div([
        html.Label("Filter by Rescue Type"),
        dcc.RadioItems(
            id='rescue-type-filter',
            options=[
                {"label": "Water Rescue", "value": "Water"},
                {"label": "Mountain or Wilderness Rescue", "value": "Mountain"},
                {"label": "Disaster or Individual Tracking", "value": "Disaster"},
                {"label": "Reset (Show All)", "value": "Reset"},
            ],
            value="Reset",  # Default selection
            inline=True
        ),
    ]),
    html.Hr(),
    
    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client

        # Pagination: Allow user to navigate pages
        page_size=10,  # Show 10 rows per page

        # Filtering: Enable users to search data in each column
        filter_action="native",  

        # Sorting: Allow sorting by clicking column headers
        sort_action="native",  

        # Row Selection: Enable single row selection (for map updates)
        row_selectable="single",

        # Styling for better readability
        style_table={'overflowX': 'auto'},  # Horizontal scroll if needed
        style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'},
        style_cell={'textAlign': 'left', 'padding': '10px'}
    ),
    html.Br(),
    html.Hr(),
     
    # Container for Pie Chart and Map (Side-by-Side)
    html.Div([
        # Pie Chart (Left)
        html.Div([
            html.H3("Breed Distribution"),
            dcc.Graph(id='pie-chart', style={'width': '100%', 'height': '400px'})
        ], style={'width': '48%', 'display': 'inline-block', 'vertical-align': 'top'}),

        # Map (Right)
        html.Div(id='map-id', className='col s12 m6',
                 style={'width': '48%', 'display': 'inline-block'})
    ], style={'display': 'flex', 'justify-content': 'space-between'}),

    html.Br(),
    html.Hr(),
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None or len(selected_columns) == 0:  # Handle NoneType case
        return []  # Return an empty list instead of causing an error
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable

# Define filtering options for Rescue Types
rescue_filter_options = {
    "Water": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"],
    "Mountain": ["German Shepherd", "Alaskan Malamute", "Siberian Husky", "Rottweiler"],
    "Disaster": ["Doberman Pinsch", "Doberman Pinsch Mix", "Bloodhound", "Golden Retriever", "Rottweiler"]
}

@app.callback(
    [Output('datatable-id', 'data'),
     Output('map-id', "children"),
     Output('pie-chart', "figure")],  # Add Pie Chart as an output
    [Input('rescue-type-filter', "value"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_dashboard(selected_rescue, index):
    # Default query (Show all dogs)
    if selected_rescue == "Reset":
        query = {"animal_type": {"$exists": True}}  # Fetch ALL records
    else:
        query = {"animal_type": "Dog"}
        if selected_rescue in rescue_filter_options:
            query["breed"] = {"$in": rescue_filter_options[selected_rescue]}

    # Fetch filtered data
    filtered_data = shelter.read(query)
    dff = pd.DataFrame(filtered_data)

    # Ensure valid DataFrame
    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)

    # Default map when data is empty
    if dff.empty:
        return dff.to_dict('records'), dl.Map(children=[
            dl.TileLayer(),
            dl.Marker(position=[30.75, -97.48], children=[
                dl.Popup(html.P("No data available"))
            ])
        ], style={'width': '1000px', 'height': '500px'}), px.pie(title="No Data Available")

    # Ensure location data is numeric
    dff['location_lat'] = pd.to_numeric(dff['location_lat'], errors='coerce')
    dff['location_long'] = pd.to_numeric(dff['location_long'], errors='coerce')
    dff = dff.dropna(subset=['location_lat', 'location_long'])

    # If no row is selected, default to the first row
    row = index[0] if index and len(index) > 0 and index[0] < len(dff) else 0

    # Extract coordinates
    try:
        lat, lon = float(dff.iloc[row]['location_lat']), float(dff.iloc[row]['location_long'])
    except (ValueError, TypeError, IndexError):
        lat, lon = 30.75, -97.48  # Default location if invalid

    # Generate Pie Chart for Breed Distribution
    breed_counts = dff['breed'].value_counts().reset_index()
    breed_counts.columns = ['Breed', 'Count']
    pie_chart = px.pie(breed_counts, values='Count', names='Breed', title='Breed Distribution')

    return dff.to_dict('records'), dl.Map(children=[
        dl.TileLayer(),
        dl.LayerGroup(id="marker-layer", children=[
            dl.Marker(position=[lat, lon], children=[
                dl.Popup([
                    html.P(f"Breed: {dff.iloc[row].get('breed', 'N/A')}"),
                    html.P(f"Animal Name: {dff.iloc[row].get('name', 'N/A')}")
                ])
            ])
        ])
    ], style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=12), pie_chart

app.run_server(debug=True)

In [ ]:
!lsof -i :27701

In [ ]:
!kill -9 522467